In [ ]:
from langgraph.graph import MessagesState
from typing_extensions import TypedDict
from typing import Optional
from langchain_core.messages import BaseMessage
from typing import Annotated
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

class ConfigSchema(TypedDict):
    model: str
    thread_id:str


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage
from langchain_core.documents import Document
from dotenv import load_dotenv

load_dotenv()

def db_generator():
    response = "Best Story: Mr Bean Story"
    #text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    #   chunk_size=100, chunk_overlap=50
    #   )
    #txt_splits = text_splitter.split_text(response)
    document = Document(
    page_content=response,
    metadata={"source": "Feedback"},
    )
    vectorstore = FAISS.from_documents([document], embedding=OpenAIEmbeddings())
    vectorstore.save_local("feedback")

In [ ]:
db_generator()

In [ ]:
from config import State
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
from pathlib import Path
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool
from retriver import get_retriever_tool
from langgraph.prebuilt import ToolNode
load_dotenv()

def story_generator(state:State,config):
    GENERATOR_PROMPTS =""" Write a short bedtime story, using simple vocabulary that is suitable for children aged 5 to 10.
                        - Make the vocabulary more age-appropriate
                        - Ensure the story is gentle, imaginative, and calming
                        - Include a simple moral
                        - Consider past feedback to guide your edits."""
    
    
    model = init_chat_model(model=str(config["configurable"]["model"]),temperature = 0.1)
    retriever_tool = get_retriever_tool()
    story_response = model.invoke(
    [{"role":"system","content":GENERATOR_PROMPTS}],
    state["messages"]).bind_tools([retriever_tool], tool_choice= "any")
    
    #print("generator",state["messages"])
    return {"messages": state["messages"] + [story_response]}




In [ ]:
from pathlib import Path
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

def get_retriever_tool():
    vectorstore = FAISS.load_local(
        "feedback",
        embeddings=OpenAIEmbeddings(),
        allow_dangerous_deserialization = True
        
    )
    retriever = vectorstore.as_retriever()

    print(retriever.invoke("some comedy sotry"))
    return create_retriever_tool(
        retriever,
        name="feedback",
        description="Use feedback from previous stories to guide improvements."
    )   

In [ ]:
retriever_tool = get_retriever_tool()

In [ ]:
retriever_tool.invoke("comedy story")

In [ ]:
retriever_tool.invoke({"query": "some comedy story"})

In [ ]:
tool = ToolNode([retriever_tool])
print(tool)

In [ ]:
from langgraph.prebuilt import ToolNode
from langchain_core.messages import AIMessage
from langchain_core.messages.tool import ToolMessage
from uuid import uuid4


retriever_tool = get_retriever_tool()

tool_node = ToolNode(tools=[retriever_tool])

tool_call_id = f"call_{uuid4().hex}" 
ai_message = AIMessage(
    content="",  
    tool_calls=[
        {
            "name": "feedback",
            "args": {"query": "can i listen to the story of an old man"},
            "id": tool_call_id,
            "type": "function",
        }
    ]
)

input_state = {"messages": [ai_message]}

output = tool_node.invoke(input_state)


print(output)


In [ ]:
from config import State
from dotenv import load_dotenv
from feedback import feedback
from langgraph.types import interrupt, Command
from langgraph.prebuilt.interrupt import HumanInterrupt
from langchain_core.messages import HumanMessage
import os
load_dotenv()

def LLMJudge(state:State):
    request: HumanInterrupt = {
        "action_request":{
                "action": "Feedback",
                "args": state["messages"]
        },
        "config": {
            "allow_ignore": True,
            "allow_respond": True,
            "allow_edit": False,
            "allow_accept": False
        },
        "description": "Please provide the Feedback"
    }
     
    response = interrupt([request])
    print(response)
    if response[0]["type"] == "response":
       return Command(goto=["feedback"],update="Feedback: "+response[0]["args"])
    if response[0]["type"] == "ignore":
        return Command(goto=["feedback"],update="Feedback: "+response[0]["args"])

In [ ]:
from config import State
from typing import Optional
from langchain.chat_models import init_chat_model
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
from langchain_core.documents import Document
load_dotenv()


def feedback(state:State,config):
    FEEDBACK_PROMPTS ="""You are a feedback summarizer for a bedtime storytelling assistant. Your role is to review the entire conversation and generate a brief summary in 3–4 sentences.
                            Instructions:
                            - If the user provided explicit feedback, identify which part of the story they referred to and summarize their likes or dislikes accordingly.                            
                            - If no feedback was provided, summarize the storyline requested and the story generated
                        """
    
   
    model = init_chat_model(model=str(config["configurable"]["model"]),temperature = 0.1)
    Judge_response = model.invoke([
        {"role": "system", "content": FEEDBACK_PROMPTS},
         state["messages"],
    ])
    #text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    #    chunk_size=100, chunk_overlap=50
    #    )
    #txt_splits = text_splitter.split_text(Judge_response.content)
    document = Document(
    page_content=Judge_response,
    metadata={"source": "Feedback"},
    )
    vectorstore = FAISS.add_documents([document], embedding=OpenAIEmbeddings())
    vectorstore.save_local("feedback")

In [10]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
load_dotenv()
vectorstore = FAISS.load_local("feedback", OpenAIEmbeddings(), allow_dangerous_deserialization=True)
all_docs = vectorstore.similarity_search(query="", k=5)

for i, doc in enumerate(all_docs):
    print(f"Doc {i+1}:\n{doc.page_content}\n---")

Doc 1:
The user is interested in fairy tales, moral stories, and animal stories.
---
Doc 2:
On 06/02/2025, no specific feedback was provided for the bedtime story generated. The story featured a curious squirrel named Lily who embarked on a magical adventure in the forest, discovering the true treasures of kindness and love.
---
Doc 3:
Based on the feedback provided on 06/02/2025, it is important to avoid repeating the character "Sammy" and the same storyline in future bedtime stories. Users have expressed a desire for variety and fresh narratives in the stories generated.
---
Doc 4:
The user provided feedback on 06/02/2025 requesting to avoid repeating the story and characters in future bedtime stories.
---
Doc 5:
Feedback received on 06/02/2025: The user liked the story as characters were not repeated.

Based on the feedback, the user appreciated the fresh characters in the story and enjoyed the narrative without character repetition.
---


In [9]:

from config import State, ConfigSchema
from langgraph.graph import END, StateGraph, START
from langchain_core.messages import HumanMessage
import os
from dotenv import load_dotenv
from IPython.display import Image, display
from langgraph.checkpoint.memory import InMemorySaver
from LLMJudge import LLMJudge
from feedback import feedback
from story_generator import story_generator
from pathlib import Path
from retriver import get_retriever_tool
from langgraph.prebuilt import ToolNode
from langchain_core.messages import convert_to_messages
from db_generate import db_generator
from langchain_core.messages import ToolMessage

load_dotenv()
feedback_path = Path("feedback")
if not feedback_path.exists(): 
    db_generator()                      

def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls:
        return "LLMJudge"
    else:
        return "retrieve"
retriever_tool = get_retriever_tool()
builder = StateGraph(State, config_schema=ConfigSchema)
config = {"configurable": {"model":  os.getenv("MODEL"), "thread_id":"story_1"}}
builder.add_node("story_generator", story_generator)
builder.add_node("retrieve",ToolNode([retriever_tool]))
builder.add_node("LLMJudge",LLMJudge)
builder.add_node("feedback", feedback)


builder.add_edge(START, "story_generator")
builder.add_conditional_edges("story_generator",should_continue, {"retrieve":"retrieve","LLMJudge":"LLMJudge"})
builder.add_edge("retrieve", "story_generator")
builder.add_edge("LLMJudge","feedback")
builder.add_edge("feedback",END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)


graph_dia = graph.get_graph().draw_mermaid_png()

with open("mermaid_diagram.png", "wb") as f:
    f.write(graph_dia)


input_state = {"messages": [HumanMessage(content="I want to listen to some best story")]}
result = graph.invoke(input_state,config=config)
     


ImportError: cannot import name 'feedback' from 'feedback' (unknown location)

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, AnyMessage
from langchain.tools.retriever import create_retriever_tool
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, List
from dotenv import load_dotenv
import os
from typing import Annotated
from langchain.chat_models import init_chat_model
load_dotenv()

from langgraph.graph.message import add_messages

from langgraph.graph import MessagesState
from typing_extensions import TypedDict
from typing import Optional

from langchain_core.messages import BaseMessage 
class State(MessagesState):
    messages: list[BaseMessage]

class ConfigSchema(TypedDict):
    model: str
    thread_id:str

# --- Step 3: Create ToolNode ---
retriever_tool = get_retriever_tool()
tool_node = ToolNode(
    tools=[retriever_tool],
    name="tools",
    handle_tool_errors=True
)

# --- Step 4: LLM node that generates tool_calls ---
def llm_node(state: State) -> State:
    model = init_chat_model(model=str(config["configurable"]["model"]),temperature = 0.1)
    model_with_tools = model.bind_tools([retriever_tool], tool_choice= "any")
    response=model_with_tools.invoke(state["messages"])
    return {"messages": state["messages"] + [response]}

# --- Step 5: Assemble the graph ---
graph = StateGraph(State)
graph.add_node("llm", llm_node)
graph.add_node("tools", tool_node)

graph.set_entry_point("llm")
graph.add_edge("llm", "tools")
graph.add_edge("tools", "llm")

checkpointer = InMemorySaver()
app = graph.compile(checkpointer=checkpointer)

# --- Step 7: Run the graph with an input message ---
input_state = {
    "messages": [HumanMessage(content="some comedy story")]
}

result = app.invoke(input_state,config=config)

# --- Step 8: Output the result ---
if __name__ == "__main__":
    print(result["messages"])


In [7]:
from datetime import datetime
import os
today_str = datetime.today().strftime("%m/%d/%Y") 
print(today_str)

06/02/2025
